Our long-term goal is to make Mojo a *superset of Python* (that is, to make Mojo 
compatible with existing Python programs). Python programmers should be able to
use Mojo immediately, and be able to access the huge ecosystem of Python 
packages that are available today. 

However, Mojo is still in early development and many Python features are not yet
implemented. You can't currently write everything in Mojo that you can write in
Python. And Mojo doesn't have its own ecosystem of packages yet.

To help bridge this gap, Mojo lets you import Python modules, call Python 
functions and interact with Python objects from Mojo code. It runs Python code
using a standard Python interpreter (CPython), so your existing Python code
doesn't need to change.

## Import a Python module

To import a Python module in Mojo, just call 
[`Python.import_module()`](/mojo/stdlib/python/python/Python#import_module) 
with the module name:

In [1]:
from python import Python

fn use_array() raises:
    # This is equivalent to Python's `import numpy as np`
    var np = Python.import_module("numpy")

    # Now use numpy as if writing in Python
    var array = np.array([1, 2, 3])
    print(array)

In [2]:
use_array()

[1 2 3]


Yes, this imports Python NumPy, and you can import *any other Python module*
that you have installed.

A few things to note:

- Currently, you cannot import individual members (such as a single Python class
  or function)—you must import the whole Python module and then access members
  through the module name.

- Mojo doesn't yet support top-level code, so the `import_module()` call must
  be inside another method. This means you may need to import a module multiple
  times or pass around a reference to the module. This works the same way as 
  Python: importing the module multiple times won't run the initialization
  logic more than once, so you don't pay any performance penalty.

- `import_module()` may raise an exception (for example, if the module isn't
  installed). If you're using it inside an `fn` function, you need to either
  handle errors (using a `try/except` clause), or add the `raises` keyword to
  the function signature. You'll also see this when calling Python functions
  that may raise exceptions. (Raising exceptions is much more common in Python
  code than in the Mojo standard library, which 
  [limits their use for performance reasons](/mojo/roadmap.html#the-standard-library-has-limited-exceptions-use).)

:::note

Mojo loads the Python interpreter and Python modules at runtime, so
wherever you run a Mojo program, it must be able to access a compatible Python
interpreter, and to locate any imported Python modules. For more information,
see [Python environment](#python-environment).

:::

### Import a local Python module

If you have some local Python code you want to use in Mojo, just add
the directory to the Python path and then import the module.

For example, suppose you have a Python file named `mypython.py`:

```{.python filename="mypython.py"}
import numpy as np

def gen_random_values(size, base):
    # generate a size x size array of random numbers between base and base+1
    random_array = np.random.rand(size, size)
    return random_array + base
```

Here's how you can import it and use it in a Mojo file:

```{.mojo filename="main.mojo"}
from python import Python

fn main() raises:
    Python.add_to_path("path/to/module")
    var mypython = Python.import_module("mypython")

    var values = mypython.gen_random_values(2, 3)
    print(values)
```

Both absolute and relative paths work with 
[`add_to_path()`](/mojo/stdlib/python/python/Python#add_to_path). For example, you
can import from the local directory like this:

```mojo
Python.add_to_path(".")
```

## Call Mojo from Python

As shown above, you can call out to Python modules from Mojo. However, there's 
currently no way to do the reverse—import Mojo modules from Python or call Mojo
functions from Python.

This may present a challenge for using certain modules. For example, many UI 
frameworks have a main event loop that makes callbacks to user-defined code
in response to UI events. This is sometimes called an "inversion of control" 
pattern. Instead of your application code calling *in* to a library, the 
framework code calls *out* to your application code.

This pattern doesn't work because you can't pass Mojo callbacks to a Python 
module.

For example, consider the popular [Tkinter package](https://docs.python.org/3/library/tkinter.html). 
The typical usage for Tkinter is something like this:

- You create a main, or "root" window for the application.
- You add one or more UI widgets to the window. The widgets can have associated
  callback functions (for example, when a button is pushed).
- You call the root window's `mainloop()` method, which listens for events, 
  updates the UI, and invokes callback functions. The main loop keeps running
  until the application exits.

Since Python can't call back into Mojo, one alternative is to have the Mojo
application drive the event loop and poll for updates. The following example
uses Tkinter, but the basic approach can be applied to other packages.

First we create a Python module that defines a Tkinter interface, with a window
and single button:

In [1]:
%%python
import tkinter as tk

class App:
    def __init__(self):
        self._root = tk.Tk()
        self.clicked = False

    def click(self):
        self.clicked = True

    def create_button(self, button_text: str):
        button = tk.Button(
            master=self._root,
            text=button_text,
            command=self.click
        )
        button.place(relx=0.5, rely=0.5, anchor=tk.CENTER)

    def create(self, res: str):
        self._root.geometry(res)
        self.create_button("Hello Mojo!")

    def update(self):
        self._root.update()

We can call this module from Mojo like this:

In [2]:
from python import Python

fn button_clicked():
    print("Hi from a Mojo🔥 fn!")

def main():
    Python.add_to_path(".")
    var app = Python.import_module("myapp").App()
    app.create("800x600")

    while True:
        app.update()
        if app.clicked:
            button_clicked()
            app.clicked = False

Instead of the Python module calling the Tkinter `mainloop()` method, the Mojo 
code calls the `update()` method in a loop and checks the `clicked` attribute 
after each update.

## Python environment

The Mojo SDK depends on an existing Python dynamic library. At runtime, Mojo
uses the first Python in the search path (`PATH`), to find an associated dynamic
Python library of the same version. This will also add any modules from the
activated virtual environment. 

### Resolving issues

Finding libpython may fail if the Python interpreter on top of `PATH` does not
have an associated dynamic library. Some Python distributions don't include the
shared library, and others only have a static library which isn't supported by
Mojo yet.

You can find a compatible Python on your system by running this Python script:

In [ ]:
import os
import subprocess

FIND_LIBPYTHON = """
import os
import sys
from pathlib import Path
from sysconfig import get_config_var
ext = "dll" if os.name == "nt" else "dylib" if sys.platform == "darwin" else "so"
binary = f"libpython{get_config_var('py_version_short')}.{ext}"
for folder in [Path(get_config_var(p)) for p in ["LIBPL", "LIBDIR"]]:
    libpython_path = folder / binary
    if libpython_path.is_file():
        print(libpython_path.resolve())
        exit(0)
exit(1)
"""
FIND_PYTHON_VER = "import sysconfig; print(sysconfig.get_python_version())"

exe_names = ["python3", "python"] + [f"python3.{i}" for i in range(8, 13)]
seen = []
executables = []

print("Mojo will attempt to use the first python executable from the top:\n")
print("vers | compat | path")
for path in os.environ["PATH"].split(":"):
    for exe in exe_names:
        full_path = os.path.join(path, exe)
        if os.path.exists(full_path):
            pyver = subprocess.check_output([full_path, "-c", FIND_PYTHON_VER], text=True).strip()
            res = subprocess.run([full_path, "-c", FIND_LIBPYTHON], text=True, capture_output=True)
            libpython = res.stdout.strip()
            if res.returncode != 0:
                print(f"{pyver:<7} no      {full_path}")
            elif libpython not in seen:
                print(f"{pyver:<7} yes     {full_path}")
                seen.append(libpython)
                executables.append(full_path)

if not executables:
    print("no compatible Python environments found")
else:
    print("\ncreate and activate a virtual environment to use a different Python version:")
    print(f"    {executables[-1]} -m venv .venv")
    print("    source .venv/bin/activate")

Which will produce output like:

```output
Mojo will attempt to use the first python executable from the top:

vers | compat | path
3.11    yes     /opt/homebrew/opt/python@3.11/libexec/bin/python3
3.12    yes     /opt/homebrew/bin/python3
3.9     yes     /usr/bin/python3

create and activate a virtual environment to use a different Python version:
    /usr/bin/python3 -m venv .venv
    source .venv/bin/activate
```

If you have no compatible environment, you can install a compatible version of
Python that includes shared libraries. Try following the instructions in [Set up
a Python environment with Conda](#set-up-a-python-environment-with-conda) to
install a virtual environment. 

### Set up a Python environment with Conda

Using a Python virtual environment such as 
[Conda](https://docs.conda.io/en/latest/) is one way to get a version of Python
that will work reliably with Mojo. It comes with the required dynamic library,
and ensures there are no conflicts with system dependencies. 

To set up a virtual environment with Conda:

1. Install Conda by following the 
   [Quick command-line install instructions](https://docs.conda.io/projects/miniconda/en/latest/#quick-command-line-install).

2. Initialize Conda for all the shells on your path:

   ```bash
   ~/miniconda3/bin/conda init --all
   ```

   Or just one at a time:

   ```bash
   ~/miniconda3/bin/conda init zsh
   ```

2. Restart your shell. 

3. Install your desired version of Python and activate the environment:

    ```bash
    conda create -n 3.10 python=3.10
    conda activate 3.10
    ```

After setting up the Conda virtual environment, you can install any Python 
packages you want to use with Mojo, with `conda install` or `pip install`. For
example:

```bash
conda install numpy
pip install pillow
```

Now whenever you `conda activate 3.10`, Mojo will be able to find any modules
you installed into that environment.

For more information on using Conda with Mojo, see 
[Using Mojo with Python](https://www.modular.com/blog/using-mojo-with-python) on
the Modular Blog.